# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

import re
import pandas as pd
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
from sklearn.externals import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('InsertTableName', engine)
df.drop('original', axis=1, inplace=True) # too many missing values!
df.dropna(inplace=True) # drop rest of missing values!
df.shape

(26207, 39)

In [3]:
# Separate Features and labels
X = df.message
Y = df.iloc[:,3:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # Detection of urls
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    
    # Normalization and Tokenizing
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def ml_pipeline():
    pipeline = Pipeline([
                         ('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs = -1)))
                        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train pipeline
pipeline = ml_pipeline()
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def clf_report(model, X_test, y_test):
    '''
    Generation of Classification Report for each Variable
    Input: Model, test data, labels
    Output: Print of Classification Report
    '''
    y_pred = model.predict(X_test)
    y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('------------------------------------------------------\n')
        print('Classification Report of Feature: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_df[column]))
        
# Run test
clf_report(pipeline, X_test, y_test)

------------------------------------------------------

Classification Report of Feature: related

             precision    recall  f1-score   support

        0.0       0.31      0.12      0.17      1249
        1.0       0.76      0.92      0.83      3954
        2.0       0.00      0.00      0.00        39

avg / total       0.65      0.72      0.67      5242

------------------------------------------------------

Classification Report of Feature: request

             precision    recall  f1-score   support

        0.0       0.84      0.98      0.90      4359
        1.0       0.46      0.10      0.17       883

avg / total       0.78      0.83      0.78      5242

------------------------------------------------------

Classification Report of Feature: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5242

-------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Parameter Tuning with GridSearch
parameters = {
              'clf__estimator__n_estimators': [10,20] #Smaller number of estimators chosen for the notebook to avoid shutdowns
            }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
# Run test
clf_report(cv, X_test, y_test)

------------------------------------------------------

Classification Report of Feature: related

             precision    recall  f1-score   support

        0.0       0.30      0.12      0.17      1249
        1.0       0.76      0.91      0.83      3954
        2.0       0.00      0.00      0.00        39

avg / total       0.64      0.72      0.67      5242

------------------------------------------------------

Classification Report of Feature: request

             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      4359
        1.0       0.37      0.08      0.13       883

avg / total       0.76      0.82      0.77      5242

------------------------------------------------------

Classification Report of Feature: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      0.99      0.99      5242

-------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
# Improvement of pipeline
def ml_pipeline_improved():
    pipeline_improved = Pipeline([
                                  ('vect', CountVectorizer(tokenizer=tokenize)),
                                  ('tfidf', TfidfTransformer()),
                                  ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier())))
                                ])
    return pipeline_improved

In [11]:
# Train & predict
pipeline = ml_pipeline_improved()
pipeline.fit(X_train, y_train)

# Evaluate
clf_report(pipeline, X_test, y_test)

------------------------------------------------------

Classification Report of Feature: related

             precision    recall  f1-score   support

        0.0       0.33      0.12      0.17      1249
        1.0       0.76      0.92      0.84      3954
        2.0       0.17      0.03      0.04        39

avg / total       0.65      0.73      0.67      5242

------------------------------------------------------

Classification Report of Feature: request

             precision    recall  f1-score   support

        0.0       0.85      0.89      0.87      4359
        1.0       0.31      0.24      0.27       883

avg / total       0.76      0.78      0.77      5242

------------------------------------------------------

Classification Report of Feature: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5242

-------

In [14]:
# CHECKPOINT: Save model
joblib.dump(pipeline, 'model.pkl')

['model.pkl']

In [15]:
# Load model
model = joblib.load('model.pkl')

In [16]:
# Parameter Tuning with GridSearch
parameters = {
              'clf__estimator__n_estimators': [10,20], #Smaller number of estimators chosen for the notebook to avoid shutdowns
              'clf__estimator__learning_rate': [1,2]
             }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1)
cv.fit(X_train, y_train)

# Evaluate
clf_report(cv, X_test, y_test)

------------------------------------------------------

Classification Report of Feature: related

             precision    recall  f1-score   support

        0.0       0.33      0.17      0.23      1249
        1.0       0.77      0.89      0.82      3954
        2.0       0.17      0.03      0.04        39

avg / total       0.66      0.71      0.68      5242

------------------------------------------------------

Classification Report of Feature: request

             precision    recall  f1-score   support

        0.0       0.85      0.88      0.86      4359
        1.0       0.29      0.25      0.27       883

avg / total       0.76      0.77      0.76      5242

------------------------------------------------------

Classification Report of Feature: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      0.99      0.99      5242

-------

### 9. Export your model as a pickle file

In [17]:
# CHECKPOINT: Save model
joblib.dump(cv, 'model_tuned.pkl')

['model_tuned.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.